In [1]:
import numpy as np
import pandas as pd

In [2]:
with open("Initial Data/final_training_data.txt","r") as f:
    data_list = f.readlines()
    data_list = data_list[1:-1]
    text = [i[:-3] for i in data_list]
    label = [i[-3:-1] for i in data_list]
data_dict = {"content":text, "label":label}
df = pd.DataFrame(data = data_dict, columns = ["content", "label"])
df["label"] = df["label"].apply(lambda x: 1 if x == "sw" else 0)
depressed = pd.DataFrame(pd.read_pickle("Initial Data/depressed.pickle"))
depressed = pd.DataFrame(depressed.content)
non_depressed = pd.DataFrame(pd.read_pickle("Initial Data/non_depressed.pickle"))
non_depressed = [comments for i in range(non_depressed.shape[0]) for comments in non_depressed.loc[i, "comments"] if len(comments)>100]
non_depressed = np.random.choice(non_depressed, 1980)
non_depressed = pd.DataFrame({"content":non_depressed})
depressed["label"] = 1
non_depressed["label"] = 0
df = pd.concat([df,depressed,non_depressed])
df = df.reset_index(drop = True)

In [20]:
#Depressed Posts
df["content"][6002]

'Just worked for a few years to save up some money. That would have helped me have an actual nest egg to fall back on and maybe my own apartment at the same time'

In [49]:
#Non-depressed Posts
df["content"][4]

'"Who would you choose?! So imagine one day you are given the chance to powerbomb any famous person through a table, who would you choose? I think my choice would be the guy who plays Vince in The Fast and the Furious. Never liked the look of him hahaha."\t'

In [50]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import regex as re

In [53]:
df["content"] = df["content"].apply(lambda x: word_tokenize(x.lower()))
#We will save this step for LSTM and BERT
df["content"] = df["content"].apply(lambda x: [w for w in x if w not in stopwords.words("english")])
df["content"] = df["content"].apply(lambda x: [w for w in x if any(t.isalpha() for t in w)])
df["content"] = df["content"].apply(lambda x: [x[i] if i!=len(x)-1 else re.sub("\W","",x[i]) for i in range(len(x))])
lemmatizer = WordNetLemmatizer()
df["content"]  = df["content"].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
df["content"]  = df["content"].apply(lambda x: " ".join(x))
df["content"] = df["content"].apply(lambda x: x if len(x)!= 0 else np.nan)
df = df.dropna()
df = df.reset_index(drop = True)

In [59]:
df.to_csv("new_data.csv")

In [60]:
df = pd.read_csv("new_data.csv",index_col=0)